In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu, sigmoid
# % matplotlib widget
import matplotlib.pyplot as plt
import os
import PIL
import PIL.Image

2023-04-13 19:20:05.013949: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-13 19:20:05.603735: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
print(tf.__version__)

2.12.0


In [3]:
batch_size = 32
img_height = 256
img_width = 256

In [4]:
import pathlib

directory = "/home/fernblade/github/chs/dataset-resized/"
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory,
    labels="inferred",
    label_mode='int',
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset="training"
)

Found 2390 files belonging to 5 classes.
Using 1912 files for training.


2023-04-13 19:20:06.622126: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-13 19:20:06.637594: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-13 19:20:06.637736: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [5]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    directory,
    labels="inferred",
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(256, 256),
    batch_size=32
)

Found 2390 files belonging to 5 classes.
Using 478 files for validation.


In [6]:
class_names = train_ds.class_names
class_names

['cardboard', 'glass', 'metal', 'paper', 'plastic']

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [7]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
validation_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [8]:
num_classes = 5
model = tf.keras.Sequential([
    # tf.keras.Input(shape=(256,256)),
    # tf.keras.layers.ZeroPadding2D(padding=(2,2)),
    # tf.keras.layers.Conv2D(filters=96,kernel_size=11,strides=4,padding='valid', activation='relu'),
    # tf.keras.layers.MaxPooling2D(pool_size=(3,3),strides=2),
    # tf.keras.layers.ZeroPadding2D(padding=(2,2)),
    # tf.keras.layers.Conv2D(filters=192,kernel_size=5,strides=1,padding='valid', activation='relu'),
    # tf.keras.layers.MaxPooling2D(pool_size=(3,3),strides=2),
    # tf.keras.layers.ZeroPadding2D(padding=(1,1)),
    # tf.keras.layers.Conv2D(filters=288,kernel_size=3,strides=1,padding='valid', activation='relu'),
    # tf.keras.layers.ZeroPadding2D(padding=(1,1)),
    # tf.keras.layers.Conv2D(filters=288,kernel_size=3,strides=1,padding='valid', activation='relu'),
    # tf.keras.layers.ZeroPadding2D(padding=(1,1)),
    # tf.keras.layers.Conv2D(filters=288,kernel_size=3,strides=1,padding='valid', activation='relu'),
    # tf.keras.layers.MaxPooling2D(pool_size=(3,3),strides=2),
    # tf.keras.layers.Dense(4096, activation='relu'),
    # tf.keras.layers.Dense(4096, activation='relu'),
    # tf.keras.layers.Dense(5)
    tf.keras.layers.Rescaling(1. / 255),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes)

])

In [9]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [20]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [10]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)
!mkdir -p saved_model
model.save('saved_model/trashnet')

Epoch 1/10


2023-04-13 19:20:10.127035: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1912]
	 [[{{node Placeholder/_0}}]]
2023-04-13 19:20:10.127402: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [1912]
	 [[{{node Placeholder/_4}}]]
2023-04-13 19:20:11.285922: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-04-13 19:20:12.386385: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7fe06c0070d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
20

60/60 [==============================] - ETA: 0s - loss: 1.5534 - accuracy: 0.2997

2023-04-13 19:20:19.270664: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [478]
	 [[{{node Placeholder/_0}}]]
2023-04-13 19:20:19.270935: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [478]
	 [[{{node Placeholder/_0}}]]


60/60 [==============================] - 10s 100ms/step - loss: 1.5534 - accuracy: 0.2997 - val_loss: 1.3881 - val_accuracy: 0.3849
Epoch 2/10
60/60 [==============================] - 4s 62ms/step - loss: 1.3802 - accuracy: 0.3923 - val_loss: 1.2972 - val_accuracy: 0.4393
Epoch 3/10
60/60 [==============================] - 4s 60ms/step - loss: 1.2501 - accuracy: 0.4791 - val_loss: 1.2366 - val_accuracy: 0.5251
Epoch 4/10
60/60 [==============================] - 4s 61ms/step - loss: 1.1165 - accuracy: 0.5544 - val_loss: 1.2759 - val_accuracy: 0.5439
Epoch 5/10
60/60 [==============================] - 4s 61ms/step - loss: 0.9610 - accuracy: 0.6172 - val_loss: 1.3571 - val_accuracy: 0.5314
Epoch 6/10
60/60 [==============================] - 4s 61ms/step - loss: 0.8292 - accuracy: 0.6736 - val_loss: 1.3925 - val_accuracy: 0.4833
Epoch 7/10
60/60 [==============================] - 4s 62ms/step - loss: 0.6147 - accuracy: 0.7746 - val_loss: 1.6317 - val_accuracy: 0.4770
Epoch 8/10
60/60 [====

INFO:tensorflow:Assets written to: saved_model/trashnet/assets


INFO:tensorflow:Assets written to: saved_model/trashnet/assets


In [ ]:
onverter = tf.lite.TFLiteConverter.from_saved_model('saved_model/trashnet')
tflite_model = converter.convert()


ith open('trashnet.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
import PIL.Image
img = tf.keras.utils.load_img('./dataset-resized/paper/paper392.jpg')
img=tf.keras.utils.img_to_array(img)

In [ ]:
img=img.reshape(-1,256,256,3)
img.shape

In [ ]:
+mdl=tf.keras.models.load_model('saved_model/trashnet')

\]'[;l,k./keras_prediction=mdl.predict(img)
print(keras_prediction)

In [ ]:
pred=(mdl.predict(val_ds))
ypred=pred.argmax(axis=-1)

In [ ]:
ypred

In [ ]:
y = np.concatenate([y for x, y in val_ds], axis=0)

In [ ]:
m=tf.keras.metrics.Accuracy()
m.update_state(y,ypred)

m.result().numpy

In [ ]:
m.result().numpy()